In [ ]:
import os
import re
import pywintypes
import win32file
import win32con
import datetime
import piexif
from PIL import Image
from PIL.ExifTags import TAGS
from hachoir.parser import createParser
from hachoir.metadata import extractMetadata

In [ ]:
# ------------------ CONFIG ------------------
ROOT_DIR = r'c:\Users\kuste\Desktop\project\Arhiv slik'
IMAGE_EXTS = ['.jpg', '.jpeg']
VIDEO_EXTS = ['.mp4']
YEAR_PATTERN = re.compile(r'\b(20[0-2][0-9]|2030)\b')
# --------------------------------------------

In [ ]:
# ---------- HELPERS ----------
def is_image(file): return any(file.lower().endswith(ext) for ext in IMAGE_EXTS)
def is_video(file): return any(file.lower().endswith(ext) for ext in VIDEO_EXTS)
def is_text_file(file): return file.lower() == "date.txt"

def get_exif_date(image_path):
    try:
        img = Image.open(image_path)
        exif_data = img._getexif()
        if exif_data:
            for tag_id, value in exif_data.items():
                if TAGS.get(tag_id, tag_id) == 'DateTimeOriginal':
                    return datetime.datetime.strptime(value, '%Y:%m:%d %H:%M:%S')
    except:
        pass
    return None

def get_video_creation_date(path):
    try:
        parser = createParser(path)
        if not parser:
            return None
        with parser:
            metadata = extractMetadata(parser)
        if metadata and metadata.has("creation_date"):
            return metadata.get("creation_date")
    except:
        pass
    return None

def parse_date_txt(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            line = f.readline().strip()
            line = line.replace('\u200e', '')  # remove LTR characters
            return datetime.datetime.strptime(line, "%A, %d %B %Y")
    except Exception as e:
        print(f"❌ Error reading date.txt in {file_path}: {e}")
    return None

def extract_year_from_folder(name):
    match = YEAR_PATTERN.search(name)
    if match:
        return datetime.datetime(int(match.group()), 5, 1, 12, 0, 0)
    return None

def set_file_times(path, dt):
    try:
        win_time = pywintypes.Time(dt)
        handle = win32file.CreateFile(
            path,
            win32con.GENERIC_WRITE,
            win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE | win32con.FILE_SHARE_DELETE,
            None,
            win32con.OPEN_EXISTING,
            win32con.FILE_ATTRIBUTE_NORMAL,
            None
        )
        win32file.SetFileTime(handle, win_time, win_time, win_time)
        handle.Close()
    except Exception as e:
        print(f"❌ Failed to set times for {path}: {e}")

def update_image_exif(image_path, dt):
    try:
        formatted = dt.strftime("%Y:%m:%d %H:%M:%S")
        exif_dict = piexif.load(image_path)
        exif_dict["Exif"][piexif.ExifIFD.DateTimeOriginal] = formatted
        exif_dict["Exif"][piexif.ExifIFD.DateTimeDigitized] = formatted
        exif_dict["0th"][piexif.ImageIFD.DateTime] = formatted
        exif_bytes = piexif.dump(exif_dict)
        piexif.insert(exif_bytes, image_path)
    except Exception as e:
        print(f"❌ Could not update EXIF for {image_path}: {e}")
# --------------------------------------------

In [ ]:
# ---------- MAIN ----------
folders_scanned = 0
files_updated = 0

for dirpath, _, filenames in os.walk(ROOT_DIR):
    folders_scanned += 1
    print(f"\n📁 Scanning folder: {dirpath}")
    txt_date = None
    image_dates = []
    video_dates = []

    # Step 1: check for date.txt
    for file in filenames:
        if is_text_file(file):
            txt_date = parse_date_txt(os.path.join(dirpath, file))
            if txt_date:
                print(f"📝 Found date.txt → {txt_date}")
            break

    # Step 2: collect media dates if no txt_date
    if not txt_date:
        for file in filenames:
            full_path = os.path.join(dirpath, file)
            if is_image(file):
                dt = get_exif_date(full_path)
                if dt:
                    image_dates.append((file, dt))
            elif is_video(file):
                dt = get_video_creation_date(full_path)
                if dt:
                    video_dates.append((file, dt))

    # Step 3: Determine fallback date
    fallback_date = None

    if txt_date:
        fallback_date = txt_date
    elif image_dates:
        fallback_date = image_dates[-1][1]
    elif video_dates:
        fallback_date = video_dates[-1][1]
    else:
        fallback_date = extract_year_from_folder(os.path.basename(dirpath))

    if not fallback_date:
        print("⚠️ No valid date found. Skipping folder.")
        continue

    # Step 4: Apply date to media files
    for file in filenames:
        full_path = os.path.join(dirpath, file)

        if is_image(file):
            if txt_date:
                update_image_exif(full_path, txt_date)
                set_file_times(full_path, txt_date)
            else:
                dt = get_exif_date(full_path) or fallback_date
                update_image_exif(full_path, dt)
                set_file_times(full_path, dt)
            print(f"✔️ Updated image: {file}")
            files_updated += 1

        elif is_video(file):
            # Note: cannot update internal media created metadata, only filesystem timestamps
            dt = txt_date or get_video_creation_date(full_path) or fallback_date
            set_file_times(full_path, dt)
            print(f"✔️ Updated video: {file}")
            files_updated += 1

        else:
            continue  # Skip non-media files

print(f"\n✅ Done. Folders scanned: {folders_scanned}, Files updated: {files_updated}")


📁 Scanning folder: c:\Users\kuste\Desktop\project\Arhiv slik
⚠️ No valid fallback. Skipping folder.

📁 Scanning folder: c:\Users\kuste\Desktop\project\Arhiv slik\!Prema gore kranjske
📸 Using last image date as fallback: 2021-09-23 20:48:37
✔ IMG_20210920_111621.jpg set to 2021-09-20 11:16:21
✔ IMG_20210920_112058.jpg set to 2021-09-20 11:20:58
✔ IMG_20210921_172137.jpg set to 2021-09-21 17:21:36
✔ IMG_20210921_173506.jpg set to 2021-09-21 17:35:06
✔ IMG_20210921_175826.jpg set to 2021-09-21 17:58:26
✔ IMG_20210923_130216.jpg set to 2021-09-23 13:02:16
✔ IMG_20210923_204824-je uredil(-a)_.jpg set to 2021-09-23 20:48:24
✔ IMG_20210923_204824.jpg set to 2021-09-23 20:48:24
✔ IMG_20210923_204837-je uredil(-a)_.jpg set to 2021-09-23 20:48:37
✔ IMG_20210923_204837.jpg set to 2021-09-23 20:48:37
✔ received_3173426426226982.jpeg set to 2021-09-23 20:48:37
✔ VID_20210922_180333~2.mp4 set to 2021-09-22 16:03:53

📁 Scanning folder: c:\Users\kuste\Desktop\project\Arhiv slik\Arhiv
📸 Using last ima